In [72]:
import pandas as pd
import json
import ast
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

## Task 3: Implementing Inter-Annotator Agreement
- #### Export the annotations in either JSON or CSV files.
> We have exported the annotations from label-studio as csv, and then __processed__ it to match the schema given as example [here](https://docs.google.com/spreadsheets/d/1yOrKVgBYuam2MqqKPXY2wMBLTlx_VtlXpUQdXXW18LM/edit?gid=1503756051#gid=1503756051) by sir.

_Example_:   
__Sentence__: इन दोनों Freight Corridor के इर्द गिर्द दिल्ली-मुंबई Industrial Corridor और अमृतसर-कोलकाता Industrial Corridor भी विकसित किए जा रहे हैं।   
__POS Tags__: [{'word': 'इन', 'entity': 'DET'}, {'word': 'दोनों', 'entity': 'DET'}, {'word': 'Freight Corridor', 'entity': 'PROPN'}, {'word': 'के', 'entity': 'ADP'}, {'word': 'इर्द गिर्द', 'entity': 'ADP'}, {'word': 'और', 'entity': 'CONJ'}, {'word': 'भी', 'entity': 'PART'}, {'word': 'विकसित', 'entity': 'VERB'}, {'word': 'किए', 'entity': 'VERB'}, {'word': 'दिल्ली-मुंबई Industrial Corridor', 'entity': 'PROPN'}, {'word': 'अमृतसर-कोलकाता Industrial Corridor', 'entity': 'PROPN'}, {'word': 'हैं', 'entity': 'NOUN'}, {'word': 'जा रहे', 'entity': 'VERB'}, {'word': '।', 'entity': 'X'}]

In [ ]:
# Load the CSV file
df = pd.read_csv("NLP_POS_480_499_Romit.csv")

# Initialize a list to store the formatted output
formatted_data = []

# Loop through each row in the dataframe
for _, row in df.iterrows():
    # Extract the sentence (from "label" column)
    labels = json.loads(row['label'])
    
    # Create the POS tag list for the sentence
    pos_tags = [{'word': item['text'], 'entity': item['labels'][0]} for item in labels]
    
    # Extract sentence text
    sentence = row['text']
    
    # Append the result to the list
    formatted_data.append([sentence, pos_tags])

# Convert the list into a new DataFrame
output_df = pd.DataFrame(formatted_data, columns=["Sentences", "POS Tags"])

# Save the result to a new CSV file
output_df.to_csv("NLP_POS_480_499_Romit_processed.csv", index=False)

print("Output saved to output.csv")

- #### Using Python code, calculate Cohen’s Kappa & Fleiss Kappa.
    - Use Cohen’s Kappa for the NLP Dataset Task.  
    - Use Fleiss Kappa for the CV Dataset Task. 
    - Get the third annotation from any other teams and then calculate the Fleiss Kappa.

> We are using sklearn's `cohen_kappa_score` function to calculate the cohen kappa between 2 arrays, which contain the tags given to the words respectively. Also, missing or unmatched phrases/words are represented by X and are present in the arrays.

In [67]:
annotations1

[[{'word': 'इन', 'entity': 'DET'},
  {'word': 'दोनों', 'entity': 'DET'},
  {'word': 'Freight Corridor', 'entity': 'PROPN'},
  {'word': 'के', 'entity': 'ADP'},
  {'word': 'इर्द गिर्द', 'entity': 'ADP'},
  {'word': 'और', 'entity': 'CONJ'},
  {'word': 'भी', 'entity': 'PART'},
  {'word': 'विकसित', 'entity': 'VERB'},
  {'word': 'किए', 'entity': 'VERB'},
  {'word': 'दिल्ली-मुंबई Industrial Corridor', 'entity': 'PROPN'},
  {'word': 'अमृतसर-कोलकाता Industrial Corridor', 'entity': 'PROPN'},
  {'word': 'हैं', 'entity': 'NOUN'},
  {'word': 'जा रहे', 'entity': 'VERB'},
  {'word': '।', 'entity': 'X'}],
 [{'word': 'उत्तर प्रदेश', 'entity': 'PROPN'},
  {'word': 'वाराणसी', 'entity': 'PROPN'},
  {'word': 'गाज़ीपुर', 'entity': 'PROPN'},
  {'word': 'Perishable Cargo Center', 'entity': 'NOUN'},
  {'word': 'और', 'entity': 'CONJ'},
  {'word': 'दो', 'entity': 'NUM'},
  {'word': 'बड़े', 'entity': 'ADJ'},
  {'word': 'किसानों', 'entity': 'NOUN'},
  {'word': 'में', 'entity': 'ADP'},
  {'word': 'में', 'entity': '

In [66]:
def make_lists(annotations1, annotations2):
  mega_list = [[], []]
  for index in range(len(annotations1)):
    word_list, tags_list = zip(*[(item['word'], item['entity']) for item in annotations1[index]])
    # print(word_list)
    word_list2, tags_list2 = zip(*[(item['word'], item['entity']) for item in annotations2[index]])
    # convert tuples to lists
    word_list = list(word_list)
    tags_list = list(tags_list)
    word_list2 = list(word_list2)
    tags_list2 = list(tags_list2)
    
    for x in range(len(word_list)):
        try:
          index = word_list2.index(word_list[x])
          rate_1 = tags_list[x]
          rate_2 = tags_list2.pop(index)
          word_list2.pop(index)
          mega_list[0].append(rate_1)
          mega_list[1].append(rate_2)

        except ValueError:
          rate_1 = tags_list[x]
          rate_2 = "X"
          mega_list[0].append(rate_1)
          mega_list[1].append(rate_2)

    for x in range(len(word_list2)):
      try:
        index = word_list.index(word_list2[x])
        rate_1 = tags_list[index]
        rate_2 = tags_list2[x]
        mega_list[0].append(rate_1)
        mega_list[1].append(rate_2)

      except ValueError:
          rate_1 = "X"
          rate_2 = tags_list2[x]
          mega_list[0].append(rate_1)
          mega_list[1].append(rate_2)


  return mega_list

In [65]:
# Load the annotations from CSV files for annotator 1 (Romit) and annotator 2 (Rudra)
annotations1 = pd.read_csv('NLP_POS_480_499_Romit.csv')['POS Tags']
annotations2 = pd.read_csv('NLP_POS_480_499_Rudra.csv')['POS Tags']

# converting the JSON representation of the POS tags to a list of python dictionaries
annotations1 = [ast.literal_eval(item) for item in annotations1]
annotations2 = [ast.literal_eval(item) for item in annotations2]

data = make_lists(annotations1, annotations2)
print("Rater A", data[0])
print("Rater B", data[1])
print("Cohen's Kappa: ", cohen_kappa_score(data[0], data[1]))

Rater A ['DET', 'DET', 'PROPN', 'ADP', 'ADP', 'CONJ', 'PART', 'VERB', 'VERB', 'PROPN', 'PROPN', 'NOUN', 'VERB', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'PROPN', 'PROPN', 'PROPN', 'NOUN', 'CONJ', 'NUM', 'ADJ', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'PART', 'ADP', 'VERB', 'VERB', 'X', 'VERB', 'VERB', 'X', 'X', 'X', 'X', 'X', 'X', 'DET', 'PROPN', 'ADP', 'NOUN', 'NOUN', 'NUM', 'ADP', 'ADP', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'PART', 'PART', 'VERB', 'VERB', 'NOUN', 'VERB', 'X', 'X', 'X', 'X', 'PROPN', 'PROPN', 'NOUN', 'ADP', 'NOUN', 'VERB', 'VERB', 'CONJ', 'ADV', 'ADV', 'ADP', 'PRON', 'VERB', 'VERB', 'VERB', 'VERB', 'X', 'X', 'X', 'X', 'VERB', 'PRON', 'ADV', 'NOUN', 'PART', 'ADP', 'PROPN', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'VERB', 'VERB', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'PROPN', 'NOUN', 'ADP', 'NOUN', 'PART', 'NOUN', 'NOUN', 'VERB', 'ADP', 'VERB', 'ADP', 'PROPN', 'ADP', 'PROPN', 'ADP',

> We got a kappa value of 0.55, which means there is considerable argument between the annotators

#### For CV, we have the dataframe with the annotator's choices (Truck or No Truck) encoded as 0 and 1 respectively

In [80]:
# for fleiss kappa on CV dataset
annotations1 = pd.read_csv("CV_annotation_480_499_Romit.csv")["annotator_choice"]
annotations2 = pd.read_csv("CV_annotation_480_499_Rudra.csv")["annotator_choice"]
annotations3 = pd.read_csv("CV_annotation_480_499_Soham.csv")["annotator_choice"]

df = pd.concat([annotations1, annotations2, annotations3], axis=1)
df = df.replace({'Trucks': 0, 'No Trucks': 1})

df_new = []
for img in df.to_numpy():
  count_trucks = 0
  count_no_trucks = 0
  for t in range(3):
    if img[t] == 0:
      count_trucks += 1
    else:
      count_no_trucks += 1

  df_new.append([count_trucks, count_no_trucks])

df = pd.DataFrame(df_new)

k = fleiss_kappa(df.to_numpy(), method='fleiss')
print("Fliess Kappa:", k)

Fliess Kappa: 0.8611111111111109


C:\Users\romit\AppData\Local\Temp\ipykernel_31308\3203798083.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'Trucks': 0, 'No Trucks': 1})


> We obtain a fleiss kappa valyue of 0.86, which means therw is almost perfect agreement between the annotators of this data